<a href="https://colab.research.google.com/github/torrozz/Graduation/blob/main/App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cài đặt streamlit và pyngrok để chạy trong Colab
!pip install -q streamlit pyngrok fastai

# Tạo file cấu hình để tránh lỗi default browser
!mkdir -p ~/.streamlit
with open("/root/.streamlit/config.toml", "w") as f:
    f.write("""
[server]
headless = true
enableCORS=false
port = 8501
""")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 739.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Copy 5 model .pkl từ MyDrive về thư mục làm việc
!cp "/content/drive/MyDrive/breast_cancer_model.pkl" /content/
!cp "/content/drive/MyDrive/oral_cancer_model.pkl" /content/
!cp "/content/drive/MyDrive/colon_model.pkl" /content/
!cp "/content/drive/MyDrive/kidney_cancer_model.pkl" /content/
!cp "/content/drive/MyDrive/metastatic_model.pkl" /content/
!cp "/content/drive/MyDrive/breast_cancer.pkl" /content/
!cp "/content/drive/MyDrive/lung_cancer_model.pkl" /content/
print("✅ Đã copy toàn bộ model vào thư mục làm việc!")

✅ Đã copy toàn bộ model vào thư mục làm việc!


In [ ]:
%%writefile app.py
import streamlit as st
from fastai.vision.all import *
from PIL import Image
import os

st.set_page_config(page_title="Ung thư AI", layout="centered")
st.title("🔬 Dự đoán ung thư và di căn từ ảnh")

# Bộ upload ảnh dùng chung
uploaded_file = st.file_uploader("📤 Tải ảnh y tế", type=["jpg", "jpeg", "png", "tif"])

# Hàm dự đoán
def predict_image(model_path, file):
    learn_inf = load_learner(model_path)
    temp_filename = "temp_img.tif"

    with open(temp_filename, "wb") as f:
        f.write(file.getvalue())

    img = PILImage.create(temp_filename)
    test_dl = learn_inf.dls.test_dl([img])

    with learn_inf.no_bar(), learn_inf.no_logging():
        preds, _ = learn_inf.get_preds(dl=test_dl)

    probs = preds[0]
    pred_idx = torch.argmax(probs).item()

    if hasattr(learn_inf.dls, 'vocab'):
        pred_class = learn_inf.dls.vocab[pred_idx]
    else:
        pred_class = str(pred_idx)

    os.remove(temp_filename)
    return pred_class, probs

# Biến lưu xác suất và trạng thái để kiểm tra di căn
cancer_probs = None
image_to_check = None  # Lưu ảnh để dùng khi chuyển sang dự đoán di căn

# --- Ung thư vú ---
if st.button("🩺 Dự đoán Ung thư vú"):
    if uploaded_file is None:
        st.warning("⚠️ Vui lòng tải ảnh lên trước.")
    else:
        st.image(uploaded_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("breast_cancer_model.pkl", uploaded_file)
        st.success(f"🧠 Dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Không Ung thư: {probs[0]:.4f}\n🔴 Có Ung thư: {probs[1]:.4f}")
        cancer_probs = probs
        image_to_check = uploaded_file  # Lưu ảnh để kiểm tra di căn

# --- Ung thư miệng ---
if st.button("🦷 Dự đoán Ung thư miệng"):
    if uploaded_file is None:
        st.warning("⚠️ Vui lòng tải ảnh lên trước.")
    else:
        st.image(uploaded_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("oral_cancer_model.pkl", uploaded_file)
        st.success(f"🔍 Dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Không Ung thư: {probs[0]:.4f}\n🔴 Có Ung thư: {probs[1]:.4f}")
        cancer_probs = probs
        image_to_check = uploaded_file  # Lưu ảnh để kiểm tra di căn

# --- Ung thư đại tràng ---
if st.button("🧬 Dự đoán Ung thư đại tràng"):
    if uploaded_file is None:
        st.warning("⚠️ Vui lòng tải ảnh lên trước.")
    else:
        st.image(uploaded_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("colon_model.pkl", uploaded_file)
        st.success(f"🧫 Kết quả dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Không Ung thư: {probs[0]:.4f}\n🔴 Có Ung thư: {probs[1]:.4f}")
        cancer_probs = probs
        image_to_check = uploaded_file  # Lưu ảnh để kiểm tra di căn

# --- Ung thư thận ---
if st.button("🧪 Dự đoán Ung thư phổi"):
    if uploaded_file is None:
        st.warning("⚠️ Vui lòng tải ảnh lên trước.")
    else:
        st.image(uploaded_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("lung_cancer_model.pkl", uploaded_file)
        st.success(f"📌 Kết quả dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Không Ung thư: {probs[0]:.4f}\n🔴 Có Ung thư: {probs[1]:.4f}")
        cancer_probs = probs
        image_to_check = uploaded_file  # Lưu ảnh để kiểm tra di căn

# # --- Di căn ---
# if cancer_probs is not None and cancer_probs[1] > cancer_probs[0]:
#     if st.button("🧠 Dự đoán Di căn"):
#         if image_to_check is None:
#             st.warning("⚠️ Vui lòng tải ảnh lên trước.")
#         else:
#             st.image(image_to_check, caption="Ảnh đã chọn", use_container_width=True)
#             pred_class, probs = predict_image("metastatic_model.pkl", image_to_check)
#             st.success(f"🧬 Kết quả dự đoán: {pred_class}")
#             st.info(f"📊 Xác suất:\n🟢 Non-Metastatic: {probs[0]:.4f}\n🔴 Metastatic: {probs[1]:.4f}")
# else:
#     if cancer_probs is not None and cancer_probs[0] > cancer_probs[1]:
#         st.warning("⚠️ Hình ảnh không có ung thư, không thể dự đoán di căn.")
# Lưu vào session_state để tránh mất khi reload
if cancer_probs is not None:
    st.session_state["cancer_probs"] = cancer_probs
    st.session_state["image_to_check"] = image_to_check

# --- Di căn ---
if "cancer_probs" in st.session_state and st.session_state["cancer_probs"][1] > st.session_state["cancer_probs"][0]:
    if st.button("🧠 Dự đoán Di căn"):
        img_file = st.session_state["image_to_check"]
        st.image(img_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("metastatic_model.pkl", img_file)
        st.success(f"🧬 Kết quả dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Non-Metastatic: {probs[0]:.4f}\n🔴 Metastatic: {probs[1]:.4f}")

elif "cancer_probs" in st.session_state and st.session_state["cancer_probs"][0] > st.session_state["cancer_probs"][1]:
    st.warning("⚠️ Hình ảnh không có ung thư, không thể dự đoán di căn.")



Writing app.py


In [ ]:
%%writefile app.py
import streamlit as st
from fastai.vision.all import *
from PIL import Image
import os

st.set_page_config(page_title="Ung thư AI", layout="centered")
st.title("🔬 Dự đoán ung thư và di căn từ ảnh")

# Bộ upload ảnh dùng chung
uploaded_file = st.file_uploader("📤 Tải ảnh y tế", type=["jpg", "jpeg", "png", "tif"])

# Hàm dự đoán
def predict_image(model_path, file):
    learn_inf = load_learner(model_path)
    temp_filename = "temp_img.tif"

    with open(temp_filename, "wb") as f:
        f.write(file.getvalue())

    img = PILImage.create(temp_filename)
    test_dl = learn_inf.dls.test_dl([img])

    with learn_inf.no_bar(), learn_inf.no_logging():
        preds, _ = learn_inf.get_preds(dl=test_dl)

    probs = preds[0]
    pred_idx = torch.argmax(probs).item()

    if hasattr(learn_inf.dls, 'vocab'):
        pred_class = learn_inf.dls.vocab[pred_idx]
    else:
        pred_class = str(pred_idx)

    os.remove(temp_filename)
    return pred_class, probs

# Biến lưu xác suất và trạng thái để kiểm tra di căn
cancer_probs = None
image_to_check = None
selected_cancer_type = None  # NEW: lưu loại ung thư được chọn

# --- Ung thư vú ---
if st.button("🩺 Dự đoán Ung thư vú"):
    if uploaded_file is None:
        st.warning("⚠️ Vui lòng tải ảnh lên trước.")
    else:
        st.image(uploaded_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("breast_cancer_model.pkl", uploaded_file)
        st.success(f"🧠 Dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Không Ung thư: {probs[0]:.4f}\n🔴 Có Ung thư: {probs[1]:.4f}")
        cancer_probs = probs
        image_to_check = uploaded_file
        selected_cancer_type = "breast"

# --- Ung thư miệng ---
if st.button("🦷 Dự đoán Ung thư miệng"):
    if uploaded_file is None:
        st.warning("⚠️ Vui lòng tải ảnh lên trước.")
    else:
        st.image(uploaded_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("oral_cancer_model.pkl", uploaded_file)
        st.success(f"🔍 Dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Không Ung thư: {probs[0]:.4f}\n🔴 Có Ung thư: {probs[1]:.4f}")
        cancer_probs = probs
        image_to_check = uploaded_file
        selected_cancer_type = "oral"

# --- Ung thư đại tràng ---
if st.button("🧬 Dự đoán Ung thư đại tràng"):
    if uploaded_file is None:
        st.warning("⚠️ Vui lòng tải ảnh lên trước.")
    else:
        st.image(uploaded_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("colon_model.pkl", uploaded_file)
        st.success(f"🧫 Kết quả dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Không Ung thư: {probs[0]:.4f}\n🔴 Có Ung thư: {probs[1]:.4f}")
        cancer_probs = probs
        image_to_check = uploaded_file
        selected_cancer_type = "colon"

# --- Ung thư phổi ---
if st.button("🧪 Dự đoán Ung thư phổi"):
    if uploaded_file is None:
        st.warning("⚠️ Vui lòng tải ảnh lên trước.")
    else:
        st.image(uploaded_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("lung_cancer_model.pkl", uploaded_file)
        st.success(f"📌 Kết quả dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Không Ung thư: {probs[0]:.4f}\n🔴 Có Ung thư: {probs[1]:.4f}")
        cancer_probs = probs
        image_to_check = uploaded_file
        selected_cancer_type = "lung"

# Lưu vào session_state
if cancer_probs is not None:
    st.session_state["cancer_probs"] = cancer_probs
    st.session_state["image_to_check"] = image_to_check
    st.session_state["selected_cancer_type"] = selected_cancer_type

# --- Di căn (chỉ cho ung thư vú) ---
if (
    "cancer_probs" in st.session_state
    and "selected_cancer_type" in st.session_state
    and st.session_state["selected_cancer_type"] == "breast"
    and st.session_state["cancer_probs"][1] > st.session_state["cancer_probs"][0]
):
    if st.button("🧠 Dự đoán Di căn"):
        img_file = st.session_state["image_to_check"]
        st.image(img_file, caption="Ảnh đã chọn", use_container_width=True)
        pred_class, probs = predict_image("metastatic_model.pkl", img_file)
        st.success(f"🧬 Kết quả dự đoán: {pred_class}")
        st.info(f"📊 Xác suất:\n🟢 Non-Metastatic: {probs[0]:.4f}\n🔴 Metastatic: {probs[1]:.4f}")

elif (
    "cancer_probs" in st.session_state
    and st.session_state["cancer_probs"][0] > st.session_state["cancer_probs"][1]
):
    st.warning("⚠️ Hình ảnh không có ung thư, không thể dự đoán di căn.")


Writing app.py


In [ ]:
!ngrok config add-authtoken 2vNHTgQSBzIYQeTDgkfwfvgvb0k_4frZVtGHvp4uLbuH5qKig

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!rm -rf ~/.streamlit
!mkdir -p ~/.streamlit


In [ ]:
with open("/root/.streamlit/config.toml", "w") as f:
    f.write("""
[server]
headless = true
port = 8501
enableCORS = false
""")


In [ ]:
!pkill streamlit
!pkill -f ngrok


In [ ]:
from pyngrok import ngrok

!streamlit run app.py &>/content/log.txt &

url = ngrok.connect(8501)
print(f"🚀 Trang web của bạn đang chạy tại: {url}")


🚀 Trang web của bạn đang chạy tại: NgrokTunnel: "https://f797-34-143-163-110.ngrok-free.app" -> "http://localhost:8501"
